<a href="https://colab.research.google.com/github/Topher254/ASA/blob/main/IATAS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from ultralytics import YOLO
import cv2
import cvzone
import math
import numpy as np
from sort import *
import requests #for http requests
import json
import time
from datetime import datetime


API_URL = "http://localhost:5002/api/ads"

def ai_model():
    start_time = time.time()
    counter = 0

    while True:
        model = YOLO('../YoloWeights/yolov8n.pt')

        classNames=['Person']
        # using images
        # results = model('./Assets/1.jpeg',show=True)
        # using webcam
        # using opencv then we put the bounding boxes
        # camera object
        # cap=cv2.VideoCapture(0)
        # # above is camera no then below the width and height
        # cap.set(3,720)
        # cap.set(4,720)
        # using saved videos
        cap=cv2.VideoCapture("../Assets/video2.mp4")
        mask = cv2.imread("../Assets/mask.png")

        # sorting instance
        tracker = Sort(max_age=20,min_hits=2,iou_threshold=0.3)

        # we draw a line across,so that if a person goes past it we count them: but later it is once we detect a persons eyes

        line_limits = [80,400,1200,400]
        total_count = []
        daily_views =0
        weekly_views=[]
        monthly_views=[]
        test_location= "Nairobi"

        while True:
            success, img = cap.read()
            # overlay the mask on the image: we imagine the mask is the camera lens width
            # print("Image shape",img.shape) #shape has issues,we had to check for width and height before masking on top
            # print("Mask shape",mask.shape)
            mask = cv2.resize(mask,(img.shape[1],img.shape[0]))
            image_Region = cv2.bitwise_and(img,mask)

            # graphiscs
            image_graphics =cv2.imread("../Assets/2.jpg",cv2.IMREAD_UNCHANGED)
            if image_graphics.shape[2] == 3:  # If it only has 3 channels (RGB)
                image_graphics = cv2.cvtColor(image_graphics, cv2.COLOR_BGR2BGRA)  # Convert to 4 channels
                # Resize by a scale factor (e.g., 50% of the original size)
                scale_percent = 50
                width = int(image_graphics.shape[1] * scale_percent / 100)
                height = int(image_graphics.shape[0] * scale_percent / 100)
                new_size = (width, height)

                # Resize image
                image_graphics = cv2.resize(image_graphics, new_size, interpolation=cv2.INTER_AREA)

                cvzone.overlayPNG(img,image_graphics,(0,0))

            results = model(image_Region,stream=True)
            # check for individual bounding boxes
            detections = np.empty((0,5))

            cv2.line(img, (line_limits[0], line_limits[1]), (line_limits[2], line_limits[3]), (0, 0, 255), 2)

            for r in results:
                boxes = r.boxes
                for box in boxes:
                    x1,y1,x2,y2 = box.xyxy[0]
                    # convert the tensor to integeres
                    x1,y1,x2,y2 = int(x1),int(y1),int(x2),int(y2)
                    # print(x1,y1,x2,y2)
                    # create rectangle
                    # cv2.rectangle(img,(x1,y1),(x2,y2),(255,0,255),3)
                    # fancy rectangle from cvzone
                    w,h=x2-x1,y2-y1

                    # confidence values
                    conf=math.ceil((box.conf[0]*200))/100

                    # then we get the class based on a dataset
                    # classname
                    class_id = int(box.cls[0])
                    if class_id==0 and conf>0.3:
                        # cvzone.cornerRect(img,(x1,y1,w,h),l=15)
                        # cvzone.putTextRect(img,f'{classNames[class_id]}{conf}',(max(0,x1),max(30,y1)),scale=0.8,thickness=2)
                        # once detected,we add it to our detection array
                        current_array = np.array([x1,y1,x2,y2,conf])
                        # do vertical stack
                        # print("detections shape:", detections.shape)
                        # print("current_array shape:", current_array.shape)
                        # current_array = current_array.reshape(1, 5)
                        detections = np.vstack((detections,current_array))

                # to count the people we have to draw a line
                # then one a person goes past the line we count them
                # each person should be assigned an Id to keep track iof them
                # within the frames we must know where our person is moving to
                # we will use a tracker,such as sort by abewley
                # https://github.com/abewley/sort


            tracker_results = tracker.update(detections)
            for result in tracker_results:
                x1,y2,x2,y2,Id = result
                x1,y1,x2,y2 = int(x1),int(y1),int(x2),int(y2)
                print(result)
                cvzone.cornerRect(img,(x1,y1,w,h),l=9,rt=1,t=2,colorC=(255,0,0),colorR=(0,255,0))
                cvzone.putTextRect(img,f'{int(Id)}',(max(0,x1),max(30,y1)),scale=2,thickness=2)
                # let's get the center point.. if it touches the line we count the person
                centerx,centery= int(x1+w/2),int(y1+h/2)
                cv2.circle(img,(centerx,centery),5,(255,0,255),cv2.FILLED)

                if line_limits[0] <centerx<line_limits[2] and line_limits[1]-50<centery<line_limits[1]+50:
                    if total_count.count(Id)==0:
                        total_count.append(Id)
                            #  debuggin count
                        #  print("total count is:",total_count)


                        cv2.line(img, (line_limits[0], line_limits[1]), (line_limits[2], line_limits[3]), (0, 255, 0), 2)

                    # check if id is alerady counted

            cvzone.putTextRect(img,f'Count:{int(len(total_count))}',(0,50),scale=2,thickness=2,colorR=(0,255,2),colorT=(255, 0, 255))

            count_in_total = int(len(total_count))
            print(f"Current Count",count_in_total)

            if time.time()-start_time >=60:
                try:
                    response = requests.post(API_URL,json={"views":count_in_total,"location":test_location})
                    print(response.status_code, response.text)
                    print(f"Sent {count_in_total} views to backend: {response.json()}")
                    start_time=time.time()

                except Exception as e:
                    print("Error Sending data:...",e)
            time.sleep(1)

            cv2.imshow("Image",img)
            # cv2.imshow("ImageRegion",image_Region)
            cv2.waitKey(1)



ai_model()


ModuleNotFoundError: No module named 'ultralytics'